In [1]:
import pandas as pd
import utils_clinical_concept_extraction
import os
import utils_general_porpose
import utils_split_dataset
from medspacy.visualization import visualize_ent
import spacy

In [2]:
current_dir = os.getcwd()
print("Current directory:", current_dir)
data = utils_general_porpose.load_json(current_dir, "/cases_controls/cases_controls_20250402_205502.json")

Current directory: /home/pajaro/compu_Pipe_V3


In [3]:
# Extract patient information
patients = utils_split_dataset.send_patient_info(data)
 
# Create a list of patients with a maximum length
patients_maxLength = utils_split_dataset.make_patient_list_with_maxlength(patients, 1000000)
    


patients info convert into sequences
patients list with max length created


In [4]:
# Load mapping and clinical NLP pipeline
umlstoicd_path = "/map/map_icd10_umls.csv"
lista_example_umls_icd = utils_general_porpose.load_mapping_icd(current_dir, umlstoicd_path)
lista_NoIdentificados, lista_identificados = utils_clinical_concept_extraction.list_codes_identified(lista_example_umls_icd)
target_rules = utils_clinical_concept_extraction.load_target_rules(lista_identificados)

mapping icd10-umls loaded
list of codes identified and not identified created
target rules loaded


In [5]:
#para cargar de forma especifica los conceptos de UMLs que fueron descomprimidos por metamorphosys en español
#la base de datos debe contener siempre el idioma ingles, en la descomprensión por medido metamorphosys
name_database = "/destination_umls_es"
target_rules = target_rules
clinical_pipe = utils_clinical_concept_extraction.load_clinical_NLPpipe(current_dir, name_database, target_rules, 0.9)

medspacy nlp-pipeline loaded


In [49]:
clinical_pipe.add_pipe("medspacy_sectionizer")

In [50]:
clinical_pipe.pipe_names

['medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_quickumls',
 'medspacy_context',
 'medspacy_sectionizer']

In [51]:
type(patients_maxLength)

list

In [52]:
patients_maxLength[:1]

[{'id_cliente': 1619,
  'label': 1.0,
  'seq': 'F03X primera visita geriatría  edad: 82 fecha de nacimiento: 6.12.1931 natural de honda vive bogotá trabajó de modista, dejó el trabajo hasta 1982  antecedentes: - dm tipo 2 en tratamiento con insulina, 21.01.14: hb glic a1: 6.3 - cardiopatía hipertensiva fevi - saos en fase de insuficencia respiratoria crónica que precisa o2 domiciliario que no utiliza de forma habitual por falta de adherencia. - hipotiroidismo en tto sustitutivo - obesidad - qx: negativos - alergicos: negativos - traumatológicos: accidente d etransito en 2010n fractura mmss derecho  -tratamiento actual: insulina nph10-0-20, enalapril 20mg 1-0-1, metoprolol 50mg 1-0-1, levotiroxina 50mcg 1-0-0.   mc: no camina  ea: paciente con cuadro de 3 años de pérdida progresiva de la capacidad d emarcha, actualmente sólo marcha intradomiciliaria con bastón de puño, a esto se ha asociado perdida de memoria reciente, desorientación temporoespacial, pérdida del interés por sus aficione

In [53]:
for patient in patients_maxLength[2:3]:
    id_cliente = patient.get("id_cliente")
    seq = patient.get("seq")
    print(id_cliente, len(seq))

7039 3943


In [54]:
for patient in patients_maxLength[2:3]:
    id_cliente = patient.get("id_cliente")
    label = patient.get("label")
    seq = patient.get("seq")

In [55]:
print(seq)

M139 ortopedia cons. externa  dr useche  paciente de 70  años con  antecedente de artritis  reumatoidea.  deormidades en articulaciones, ya recibio manejo con reemplazo total de rodillas bilateral con adeucada evolucion consulta por dolro y limitacion para la movilidad de cadera , predominio derecho , alexamen fisico en cadera derecha deformidad en flexion  , retraccion muscular, roce articularm bloqueo para rotaciones y limitacion en arco de movilidad, rx y tac cadera con cambios artrosicos severos con colapsoarticular deformidad en cuello y cabeza femoral.. se considera paciente se benefica de manejo con reemplazo total de caderaderecha, ss valoracion por anestesiologia    ss rx pelvis.   ss valoracion prequrirugica por anestesia ss rx hombros codos, muñecas y manos bilateral,  , rx cervical.   ss hemograma rx torax , electrocardiograma ss valoracion por reumatologia para manejo de enfermedad de base ss procedimientos quirurgicos y material , ss protesis amarey cementada y no cementa

In [59]:
doc = clinical_pipe(seq)

for ent in doc.ents:
    print('Entity text : {}'.format(ent.text))
    print('Label (UMLS CUI) : {}'.format(ent.label_))
    print('Similarity : {}'.format(ent._.similarity))
    print('Semtypes : {}'.format(ent._.semtypes))

Entity text : M139
Label (UMLS CUI) : icd_husi
Similarity : -1.0
Semtypes : -1.0
Entity text : artritis
Label (UMLS CUI) : C0003864
Similarity : 1.0
Semtypes : {'T047'}
Entity text : movilidad
Label (UMLS CUI) : C0425245
Similarity : 1.0
Semtypes : {'T033'}
Entity text : bloqueo
Label (UMLS CUI) : C0028778
Similarity : 1.0
Semtypes : {'T033'}
Entity text : movilidad
Label (UMLS CUI) : C0425245
Similarity : 1.0
Semtypes : {'T033'}
Entity text : anestesia
Label (UMLS CUI) : C0278134
Similarity : 1.0
Semtypes : {'T184'}
Entity text : enfermedad
Label (UMLS CUI) : C0012634
Similarity : 1.0
Semtypes : {'T047'}
Entity text : M069
Label (UMLS CUI) : icd_husi
Similarity : -1.0
Semtypes : -1.0
Entity text : escolaridad
Label (UMLS CUI) : C0013658
Similarity : 1.0
Semtypes : {'T033'}
Entity text : motivo de consulta
Label (UMLS CUI) : C5441521
Similarity : 1.0
Semtypes : {'T033'}
Entity text : artritis
Label (UMLS CUI) : C0003864
Similarity : 1.0
Semtypes : {'T047'}
Entity text : enfermedad
Labe

In [68]:
doc = clinical_pipe(seq)
#cat_sem = [{"T047"}, {"T033"}, {'T184'}]
cat_sem = [{"T047"}, {'T184'}]
matches = [ent for ent in doc.ents if ent._.semtypes in cat_sem]
for ent in matches:
    print('Entity text : {}'.format(ent.text))
    print('Label (UMLS CUI) : {}'.format(ent.label_))
    print('Similarity : {}'.format(ent._.similarity))
    print('Semtypes : {}'.format(ent._.semtypes))

"""
for ent in doc.ents:
    if ent._.semtypes == {'T047'}:
        print('Entity text : {}'.format(ent.text))
        print('Label (UMLS CUI) : {}'.format(ent.label_))
        print('Similarity : {}'.format(ent._.similarity))
        
        #print('Semtypes : {}'.format(ent._.semtypes))
"""

Entity text : artritis
Label (UMLS CUI) : C0003864
Similarity : 1.0
Semtypes : {'T047'}
Entity text : anestesia
Label (UMLS CUI) : C0278134
Similarity : 1.0
Semtypes : {'T184'}
Entity text : enfermedad
Label (UMLS CUI) : C0012634
Similarity : 1.0
Semtypes : {'T047'}
Entity text : artritis
Label (UMLS CUI) : C0003864
Similarity : 1.0
Semtypes : {'T047'}
Entity text : enfermedad
Label (UMLS CUI) : C0012634
Similarity : 1.0
Semtypes : {'T047'}
Entity text : artritis reumatoide
Label (UMLS CUI) : C0003873
Similarity : 1.0
Semtypes : {'T047'}
Entity text : dolor
Label (UMLS CUI) : C0030193
Similarity : 1.0
Semtypes : {'T184'}
Entity text : dolor
Label (UMLS CUI) : C0030193
Similarity : 1.0
Semtypes : {'T184'}
Entity text : enfermedad
Label (UMLS CUI) : C0012634
Similarity : 1.0
Semtypes : {'T047'}
Entity text : dificultad respiratoria
Label (UMLS CUI) : C0013404
Similarity : 1.0
Semtypes : {'T184'}
Entity text : colon irritable
Label (UMLS CUI) : C0022104
Similarity : 1.0
Semtypes : {'T047'

"\nfor ent in doc.ents:\n    if ent._.semtypes == {'T047'}:\n        print('Entity text : {}'.format(ent.text))\n        print('Label (UMLS CUI) : {}'.format(ent.label_))\n        print('Similarity : {}'.format(ent._.similarity))\n        \n        #print('Semtypes : {}'.format(ent._.semtypes))\n"

In [57]:
visualize_ent(doc)

In [10]:
for ent in doc.ents:
    if any([ent._.is_negated, ent._.is_uncertain, ent._.is_historical, ent._.is_family, ent._.is_hypothetical, ]):
        print("'{0}' modified by {1} in: '{2}'".format(ent, ent._.modifiers, ent.sent))
        print()

'cardiopatía hipertensiva' modified by (<ConTextModifier> [28, 29, HISTORICAL],) in: 'F03X primera visita geriatría  edad: 82 fecha de nacimiento: 6.12.1931 natural de honda vive bogotá trabajó de modista, dejó el trabajo hasta 1982  antecedentes: - dm tipo 2 en tratamiento con insulina, 21.01.14: hb glic a1: 6.3 - cardiopatía hipertensiva fevi - saos en fase de insuficencia respiratoria crónica que precisa o2 domiciliario que no utiliza de forma habitual por falta de adherencia. - hipotiroidismo en tto sustitutivo - obesidad - qx: negativos - alergicos: negativos - traumatológicos: accidente d etransito en 2010n fractura mmss derecho  -tratamiento actual: insulina nph10-0-20, enalapril 20mg 1-0-1, metoprolol 50mg 1-0-1, levotiroxina 50mcg 1-0-0.   mc: no camina  ea: paciente con cuadro de 3 años de pérdida progresiva de la capacidad d emarcha, actualmente sólo marcha intradomiciliaria con bastón de puño, a esto se ha asociado perdida de memoria reciente, desorientación temporoespacial

In [11]:
for ent in doc.ents:
    if any([ent._.is_negated]):
        print("'{0}' modified by {1} in: '{2}'".format(ent, ent._.modifiers, ent.sent))
        print()

'adherencia' modified by (<ConTextModifier> [28, 29, HISTORICAL], <ConTextModifier> [69, 71, NEGATED_EXISTENCE], <ConTextModifier> [122, 124, NEGATED_EXISTENCE]) in: 'F03X primera visita geriatría  edad: 82 fecha de nacimiento: 6.12.1931 natural de honda vive bogotá trabajó de modista, dejó el trabajo hasta 1982  antecedentes: - dm tipo 2 en tratamiento con insulina, 21.01.14: hb glic a1: 6.3 - cardiopatía hipertensiva fevi - saos en fase de insuficencia respiratoria crónica que precisa o2 domiciliario que no utiliza de forma habitual por falta de adherencia. - hipotiroidismo en tto sustitutivo - obesidad - qx: negativos - alergicos: negativos - traumatológicos: accidente d etransito en 2010n fractura mmss derecho  -tratamiento actual: insulina nph10-0-20, enalapril 20mg 1-0-1, metoprolol 50mg 1-0-1, levotiroxina 50mcg 1-0-0.   mc: no camina  ea: paciente con cuadro de 3 años de pérdida progresiva de la capacidad d emarcha, actualmente sólo marcha intradomiciliaria con bastón de puño, 

In [22]:
doc._.section_titles

[]